# 네이버 cloud flatform에서 지도 API 제공

In [1]:
import folium
#import pandas as pd
import urllib.request
import datetime
import time
import json
from config_NaverAPI_okcy import *    # NAVER Client ID/Secret

In [2]:
import os
import sys
import webbrowser

## 네이버 지도 API-id, secret-key

In [3]:
#[CODE 1]
def get_request_url(url):
    
    req = urllib.request.Request(url)
    req.add_header("X-NCP-APIGW-API-KEY-ID", client_id)
    req.add_header("X-NCP-APIGW-API-KEY", client_secret)
    try: 
        response = urllib.request.urlopen(req)
        if response.getcode() == 200:
            print ("[%s] Url Request Success" % datetime.datetime.now())
            return response.read().decode('utf-8')
    except Exception as e:
        print(e)
        print("[%s] Error for URL : %s" % (datetime.datetime.now(), url))
        return None

In [4]:
base = "https://naveropenapi.apigw.ntruss.com/map-geocode/v2/geocode"
address = "울산시 남구 대학로 93"
parameters = "?query=%s" % urllib.parse.quote(address)
url = base + parameters
retData = get_request_url(url)

[2020-11-29 18:14:09.020526] Url Request Success


In [5]:
jsonAddress = json.loads(retData)
jsonAddress

{'status': 'OK',
 'meta': {'totalCount': 1, 'page': 1, 'count': 1},
 'addresses': [{'roadAddress': '울산광역시 남구 대학로 93 울산대학교',
   'jibunAddress': '울산광역시 남구 무거동 665 울산대학교',
   'englishAddress': '93, Daehak-ro, Nam-gu, Ulsan, Republic of Korea',
   'addressElements': [{'types': ['SIDO'],
     'longName': '울산광역시',
     'shortName': '울산광역시',
     'code': ''},
    {'types': ['SIGUGUN'], 'longName': '남구', 'shortName': '남구', 'code': ''},
    {'types': ['DONGMYUN'], 'longName': '무거동', 'shortName': '무거동', 'code': ''},
    {'types': ['RI'], 'longName': '', 'shortName': '', 'code': ''},
    {'types': ['ROAD_NAME'],
     'longName': '대학로',
     'shortName': '대학로',
     'code': ''},
    {'types': ['BUILDING_NUMBER'],
     'longName': '93',
     'shortName': '93',
     'code': ''},
    {'types': ['BUILDING_NAME'],
     'longName': '울산대학교',
     'shortName': '울산대학교',
     'code': ''},
    {'types': ['LAND_NUMBER'],
     'longName': '665',
     'shortName': '665',
     'code': ''},
    {'types': ['POSTAL

In [6]:
longitude = jsonAddress['addresses'][0]['x']
latitude = jsonAddress['addresses'][0]['y']
print([latitude, longitude])

['35.5437482', '129.2561333']


## 검색 주소의 지도 정보를JSON 형식으로 return

In [7]:
#[CODE 2]
def getGeoData(address):
    
    base = "https://naveropenapi.apigw.ntruss.com/map-geocode/v2/geocode"
    
    try:
        parameters = "?query=%s" % urllib.parse.quote(address)
    except:
        return None
    
    url = base + parameters
    
    retData = get_request_url(url)
    if retData == None:
        return None

    jsonAddress = json.loads(retData)

    if 'addresses' in jsonAddress.keys():
        if jsonAddress['addresses'] != []:
            latitude = jsonAddress['addresses'][0]['y']
            longitude = jsonAddress['addresses'][0]['x']
        else:
            return None
    else:
        return None
    
    return [latitude, longitude]

## folium map 객체 생성(울산대학교 )

In [10]:
map_uou = folium.Map(location=[35.5437411,129.2540956], zoom_start=12)
map_uou

In [12]:
folium.Marker([35.5437411,129.2540956], popup='울산대학교').add_to(map_uou)
map_uou

## 가맹점 데이터 파일에서 주소

In [13]:
map_chicken = folium.Map(location=[35.5437411,129.2540956], zoom_start=12)
map_chicken

In [14]:
#[CODE 4]
import pdb
filename = 'chicken_data/cheogajip_table.csv'
df = pd.read_csv(filename, encoding='CP949', index_col=0, header=0)
geoData = []

for index, row in df.iterrows():
    if row['sido'] == '울산광역시':
        geoData = getGeoData(row['store_address'])
        if geoData != None:
            folium.Marker(geoData, popup=row['store'], icon=folium.Icon(color='red')).add_to(map_chicken)

[2020-11-29 18:16:21.737999] Url Request Success
[2020-11-29 18:16:21.895514] Url Request Success
[2020-11-29 18:16:22.028223] Url Request Success
[2020-11-29 18:16:22.138752] Url Request Success
[2020-11-29 18:16:22.269009] Url Request Success
[2020-11-29 18:16:22.369959] Url Request Success
[2020-11-29 18:16:22.464624] Url Request Success
[2020-11-29 18:16:22.577009] Url Request Success
[2020-11-29 18:16:22.737096] Url Request Success
[2020-11-29 18:16:22.856136] Url Request Success
[2020-11-29 18:16:22.945581] Url Request Success
[2020-11-29 18:16:23.052947] Url Request Success
[2020-11-29 18:16:23.153384] Url Request Success
[2020-11-29 18:16:23.280780] Url Request Success
[2020-11-29 18:16:23.405950] Url Request Success
[2020-11-29 18:16:23.503989] Url Request Success
[2020-11-29 18:16:23.619632] Url Request Success
[2020-11-29 18:16:23.773003] Url Request Success
[2020-11-29 18:16:23.905342] Url Request Success
[2020-11-29 18:16:23.995390] Url Request Success
[2020-11-29 18:16:24

In [15]:
map_chicken

In [16]:
filename = 'chicken_data/kyochon_table.csv'
df = pd.read_csv(filename, encoding='CP949', index_col=0, header=0)
geoData = []
for index, row in df.iterrows():
    if row['sido'] == '울산광역시':
        geoData = getGeoData(row['store_address'])
        if geoData != None:
            folium.Marker(geoData, popup=row['store'], icon=folium.Icon(color='blue')).add_to(map_chicken)

[2020-11-29 18:16:30.972785] Url Request Success
[2020-11-29 18:16:31.064662] Url Request Success
[2020-11-29 18:16:31.167026] Url Request Success
[2020-11-29 18:16:31.271090] Url Request Success
[2020-11-29 18:16:31.378017] Url Request Success
[2020-11-29 18:16:31.467671] Url Request Success
[2020-11-29 18:16:31.561068] Url Request Success
[2020-11-29 18:16:31.809483] Url Request Success
[2020-11-29 18:16:31.913727] Url Request Success
[2020-11-29 18:16:32.009744] Url Request Success
[2020-11-29 18:16:32.116637] Url Request Success
[2020-11-29 18:16:32.217196] Url Request Success
[2020-11-29 18:16:32.320067] Url Request Success
[2020-11-29 18:16:32.406040] Url Request Success
[2020-11-29 18:16:32.499331] Url Request Success
[2020-11-29 18:16:32.635264] Url Request Success
[2020-11-29 18:16:32.740606] Url Request Success
[2020-11-29 18:16:32.836378] Url Request Success
[2020-11-29 18:16:32.930764] Url Request Success
[2020-11-29 18:16:33.021210] Url Request Success
[2020-11-29 18:16:33

In [17]:
map_chicken

In [18]:
filename = 'chicken_data/goobne_table.csv'
df = pd.read_csv(filename, encoding='CP949', index_col=0, header=0)
geoData = []
for index, row in df.iterrows():
    if row['sido'] == '울산광역시':
        geoData = getGeoData(row['store_address'])
        if geoData != None:
            folium.Marker(geoData, popup=row['store'], icon=folium.Icon(color='gray')).add_to(map_chicken)

[2020-11-29 18:16:38.168589] Url Request Success
[2020-11-29 18:16:38.269008] Url Request Success
[2020-11-29 18:16:38.371754] Url Request Success
[2020-11-29 18:16:38.508589] Url Request Success
[2020-11-29 18:16:38.689142] Url Request Success
[2020-11-29 18:16:38.792736] Url Request Success
[2020-11-29 18:16:38.889182] Url Request Success
[2020-11-29 18:16:38.981358] Url Request Success
[2020-11-29 18:16:39.079488] Url Request Success
[2020-11-29 18:16:39.188327] Url Request Success
[2020-11-29 18:16:39.326888] Url Request Success
[2020-11-29 18:16:39.437330] Url Request Success
[2020-11-29 18:16:39.534351] Url Request Success
[2020-11-29 18:16:39.676994] Url Request Success
[2020-11-29 18:16:39.819862] Url Request Success
[2020-11-29 18:16:39.925016] Url Request Success
[2020-11-29 18:16:40.020062] Url Request Success
[2020-11-29 18:16:40.126750] Url Request Success
[2020-11-29 18:16:40.259254] Url Request Success
[2020-11-29 18:16:40.346005] Url Request Success
[2020-11-29 18:16:40

In [19]:
map_chicken

In [23]:
filename = 'chicken_data/nene_table.csv'
df = pd.read_csv(filename, encoding='CP949', index_col=0, header=0)
geoData = []
for index, row in df.iterrows():
    if row['sido'] == '울산광역시':
        geoData = getGeoData(row['store_address'])
        if geoData != None:
            folium.Marker(geoData, popup=row['store'], icon=folium.Icon(color='blacks')).add_to(map_chicken)

[2020-11-29 18:17:17.985738] Url Request Success
[2020-11-29 18:17:18.086657] Url Request Success
[2020-11-29 18:17:18.177994] Url Request Success
[2020-11-29 18:17:18.274584] Url Request Success
[2020-11-29 18:17:18.370306] Url Request Success
[2020-11-29 18:17:18.479003] Url Request Success
[2020-11-29 18:17:18.574822] Url Request Success
[2020-11-29 18:17:18.694995] Url Request Success
[2020-11-29 18:17:18.796013] Url Request Success
[2020-11-29 18:17:18.886017] Url Request Success
[2020-11-29 18:17:18.992006] Url Request Success
[2020-11-29 18:17:19.085873] Url Request Success
[2020-11-29 18:17:19.172852] Url Request Success
[2020-11-29 18:17:19.264936] Url Request Success
[2020-11-29 18:17:19.356343] Url Request Success
[2020-11-29 18:17:19.451984] Url Request Success
[2020-11-29 18:17:19.547004] Url Request Success
[2020-11-29 18:17:19.706239] Url Request Success
[2020-11-29 18:17:19.798322] Url Request Success
[2020-11-29 18:17:19.892000] Url Request Success
[2020-11-29 18:17:19

In [24]:
map_chicken

In [25]:
filename = 'chicken_data/pericana_table.csv'
df = pd.read_csv(filename, encoding='CP949', index_col=0, header=0)
geoData = []
for index, row in df.iterrows():
    if row['sido'] == '울산광역시':
        geoData = getGeoData(row['store_address'])
        if geoData != None:
            folium.Marker(geoData, popup=row['store'], icon=folium.Icon(color='Greens')).add_to(map_chicken)

[2020-11-29 18:17:29.092319] Url Request Success
[2020-11-29 18:17:29.182235] Url Request Success
[2020-11-29 18:17:29.282006] Url Request Success
[2020-11-29 18:17:29.374775] Url Request Success
[2020-11-29 18:17:29.477476] Url Request Success
[2020-11-29 18:17:29.567234] Url Request Success
[2020-11-29 18:17:29.751166] Url Request Success
[2020-11-29 18:17:29.836991] Url Request Success
[2020-11-29 18:17:29.940189] Url Request Success
[2020-11-29 18:17:30.036272] Url Request Success
[2020-11-29 18:17:30.137583] Url Request Success
[2020-11-29 18:17:30.271031] Url Request Success
[2020-11-29 18:17:30.360929] Url Request Success
[2020-11-29 18:17:30.487733] Url Request Success
[2020-11-29 18:17:30.589422] Url Request Success
[2020-11-29 18:17:30.715036] Url Request Success
[2020-11-29 18:17:30.850812] Url Request Success
[2020-11-29 18:17:30.944318] Url Request Success
[2020-11-29 18:17:31.033422] Url Request Success
[2020-11-29 18:17:31.147994] Url Request Success


In [27]:
map_chicken

In [ ]:
#[CODE 5]
svFilename = 'chicken_가맹점_울산광역시.html'
map.save(svFilename)
webbrowser.open(svFilename)